In [494]:
import numpy as np
import pandas as pd
import unicodedata

In [495]:
# reading in previous datasets
mvps_df = pd.read_csv("mvp_votes.csv")
per_game_df = pd.read_csv("per_game_stats.csv")
advanced_stats_df = pd.read_csv("advanced_stats.csv")
team_records = pd.read_csv("records.csv")

# Cleaning the data

In [496]:
mvps_df = mvps_df[["Player", "Year", "Pts Won", "Pts Max", "Share"]]
mvps_df

,Player,Year,Pts Won,Pts Max,Share
0,Shaquille O'Neal,2000,1207.0,1210,0.998
1,Kevin Garnett,2000,408.0,1210,0.337
2,Alonzo Mourning,2000,367.0,1210,0.303
3,Karl Malone,2000,312.0,1210,0.258
4,Tim Duncan,2000,248.0,1210,0.205
...,...,...,...,...,...
334,Stephen Curry,2023,5.0,1000,0.005
335,Jimmy Butler,2023,3.0,1000,0.003
336,De'Aaron Fox,2023,2.0,1000,0.002
337,Jalen Brunson,2023,1.0,1000,0.001


In [497]:
per_game_df = per_game_df.drop(["Unnamed: 0", "Rk"], axis=1)
per_game_df["Player"] = per_game_df["Player"].str.replace("*","",regex=False)

In [498]:
# function to remove duplicate rows in the same year 
# because of players playing for multiple teams in the same year
# if that's the case
def single_row(group):
    if group.shape[0]==1:
        return group
    else:
        row = group[group["Tm"] == "TOT"]
        row["Tm"] = group.iloc[-1,:]["Tm"]
        return row


per_game_df = per_game_df.groupby(["Player", "Year"]).apply(single_row)

In [499]:
# run this twice to get rid of the player and year indexing caused by the groupby
per_game_df.index = per_game_df.index.droplevel()
per_game_df.index = per_game_df.index.droplevel()

In [500]:
# Looking at null values in dataset
per_game_df.isna().sum()

Player       0
Pos          0
Age          0
Tm           0
G            0
GS           0
MP           0
FG           0
FGA          0
FG%         45
3P           0
3PA          0
3P%       1484
2P           0
2PA          0
2P%         87
eFG%        45
FT           0
FTA          0
FT%        446
ORB          0
DRB          0
TRB          0
AST          0
STL          0
BLK          0
TOV          0
PF           0
PTS          0
Year         0
dtype: int64

In [501]:
# Filling in all null values for per game dataframe since they relate to percentages and those null values
# represent players who did not take those shots
per_game_df[per_game_df['FG%'].isna()]
per_game_df['FG%'] = per_game_df['FG%'].fillna(0)
per_game_df['3P%'] = per_game_df['3P%'].fillna(0)
per_game_df['2P%'] = per_game_df['2P%'].fillna(0)
per_game_df['eFG%'] = per_game_df['eFG%'].fillna(0)
per_game_df['FT%'] = per_game_df['FT%'].fillna(0)

In [502]:
advanced_stats_df.columns

Index(['Unnamed: 0', 'Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48',
       'Unnamed: 24', 'OBPM', 'DBPM', 'BPM', 'VORP', 'Year'],
      dtype='object')

In [503]:
# Dropping weird columns
advanced_stats_df = advanced_stats_df.drop(['Unnamed: 0', 'Rk', 'Unnamed: 19', 'Unnamed: 24'], axis=1)

In [504]:
advanced_stats_df.isna().sum()

Player     0
Pos        0
Age        0
Tm         0
G          0
MP         0
PER        5
TS%       80
3PAr      85
FTr       85
ORB%       5
DRB%       5
TRB%       5
AST%       5
STL%       5
BLK%       5
TOV%      67
USG%       5
OWS        0
DWS        0
WS         0
WS/48      5
OBPM       0
DBPM       0
BPM        0
VORP       0
Year       0
dtype: int64

In [505]:
# first line drops the 5 rows which are identical. represents players 
# fill na values with 0 since it references players who barely played
advanced_stats_df = advanced_stats_df.dropna(subset=['PER', 'WS/48'])
advanced_stats_df['TS%'] = advanced_stats_df['TS%'].fillna(0)
advanced_stats_df['3PAr'] = advanced_stats_df['3PAr'].fillna(0)
advanced_stats_df['FTr'] = advanced_stats_df['FTr'].fillna(0)
advanced_stats_df['TOV%'] = advanced_stats_df['TOV%'].fillna(0)

In [506]:
advanced_stats_df = advanced_stats_df.groupby(["Player", "Year"]).apply(single_row)

In [507]:
advanced_stats_df.index = advanced_stats_df.index.droplevel()
advanced_stats_df.index = advanced_stats_df.index.droplevel()

In [508]:
# Fixing asterisk problem in this dataset as well
advanced_stats_df["Player"] = advanced_stats_df["Player"].str.replace("*","",regex=False)

In [509]:
# Merge advanced stats with MVP's to get the advanced stats necessary
advanced_mvp = advanced_stats_df.merge(mvps_df, how="outer", on=["Player", "Year"])
advanced_mvp

,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,...,WS,WS/48,OBPM,DBPM,BPM,VORP,Year,Pts Won,Pts Max,Share
0,A.C. Green,PF,36,LAL,82,1929,11.2,0.482,0.010,0.245,...,5.0,0.124,-1.1,0.8,-0.3,0.8,2000,NaN,NaN,NaN
1,A.C. Green,PF,37,MIA,82,1411,11.2,0.492,0.019,0.343,...,3.2,0.110,-2.2,0.0,-2.2,-0.1,2001,NaN,NaN,NaN
2,A.J. Bramlett,C,23,CLE,8,61,-0.4,0.190,0.000,0.000,...,-0.2,-0.129,-9.2,-6.7,-15.9,-0.2,2000,NaN,NaN,NaN
3,A.J. Green,SG,23,MIL,35,345,11.5,0.607,0.840,0.032,...,0.8,0.111,-0.3,-0.6,-0.9,0.1,2023,NaN,NaN,NaN
4,A.J. Guyton,PG,22,CHI,33,630,10.3,0.495,0.359,0.094,...,0.3,0.020,-0.8,-3.2,-4.0,-0.3,2001,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11511,Željko Rebrača,C,29,DET,74,1179,16.2,0.569,0.000,0.468,...,3.3,0.134,-2.9,0.6,-2.3,-0.1,2002,NaN,NaN,NaN
11512,Željko Rebrača,C,30,DET,30,488,14.0,0.596,0.000,0.331,...,1.3,0.133,-2.8,0.2,-2.6,-0.1,2003,NaN,NaN,NaN
11513,Željko Rebrača,C,31,ATL,24,273,10.5,0.504,0.000,0.390,...,0.5,0.097,-4.5,0.6,-3.9,-0.1,2004,NaN,NaN,NaN
11514,Željko Rebrača,C,32,LAC,58,928,14.0,0.625,0.000,0.363,...,2.4,0.122,-1.7,0.4,-1.3,0.2,2005,NaN,NaN,NaN


In [510]:
# Deleting duplicate columns
advanced_mvp = advanced_mvp.drop(columns=["Pos", "Age", "Tm", "G", "MP"])

In [511]:
# Now need to merge advanced and mvp's with per game
combined = per_game_df.merge(advanced_mvp, how="outer", on=["Player", "Year"])
combined

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Pts Won,Pts Max,Share
0,A.C. Green,PF,36,LAL,82,82,23.5,2.1,4.7,0.447,...,3.3,5.0,0.124,-1.1,0.8,-0.3,0.8,NaN,NaN,NaN
1,A.C. Green,PF,37,MIA,82,1,17.2,1.8,4.0,0.444,...,2.1,3.2,0.110,-2.2,0.0,-2.2,-0.1,NaN,NaN,NaN
2,A.J. Bramlett,C,23,CLE,8,0,7.6,0.5,2.6,0.190,...,0.1,-0.2,-0.129,-9.2,-6.7,-15.9,-0.2,NaN,NaN,NaN
3,A.J. Green,SG,23,MIL,35,1,9.9,1.5,3.6,0.424,...,0.3,0.8,0.111,-0.3,-0.6,-0.9,0.1,NaN,NaN,NaN
4,A.J. Guyton,PG,22,CHI,33,8,19.1,2.4,5.8,0.406,...,-0.1,0.3,0.020,-0.8,-3.2,-4.0,-0.3,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11514,Željko Rebrača,C,29,DET,74,4,15.9,2.6,5.1,0.505,...,1.9,3.3,0.134,-2.9,0.6,-2.3,-0.1,NaN,NaN,NaN
11515,Željko Rebrača,C,30,DET,30,12,16.3,2.7,4.8,0.552,...,0.7,1.3,0.133,-2.8,0.2,-2.6,-0.1,NaN,NaN,NaN
11516,Željko Rebrača,C,31,ATL,24,2,11.4,1.4,3.2,0.442,...,0.4,0.5,0.097,-4.5,0.6,-3.9,-0.1,NaN,NaN,NaN
11517,Željko Rebrača,C,32,LAC,58,2,16.0,2.3,4.0,0.568,...,0.9,2.4,0.122,-1.7,0.4,-1.3,0.2,NaN,NaN,NaN


In [512]:
# Dropping rows where there are no advanced stats (probably irrelevant for MVP)
combined = combined.dropna(subset=['PER', 'WS/48'])

In [513]:
# Filling in 0's for those who did not get any MVP Pts Won
combined[["Pts Won", "Pts Max", "Share"]] = combined[["Pts Won", "Pts Max", "Share"]].fillna(0)

/var/folders/64/5vzyghmd6xs0bzvv0yyj64sc0000gr/T/ipykernel_20783/2729997011.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined[["Pts Won", "Pts Max", "Share"]] = combined[["Pts Won", "Pts Max", "Share"]].fillna(0)


In [514]:
combined

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Pts Won,Pts Max,Share
0,A.C. Green,PF,36,LAL,82,82,23.5,2.1,4.7,0.447,...,3.3,5.0,0.124,-1.1,0.8,-0.3,0.8,0.0,0.0,0.0
1,A.C. Green,PF,37,MIA,82,1,17.2,1.8,4.0,0.444,...,2.1,3.2,0.110,-2.2,0.0,-2.2,-0.1,0.0,0.0,0.0
2,A.J. Bramlett,C,23,CLE,8,0,7.6,0.5,2.6,0.190,...,0.1,-0.2,-0.129,-9.2,-6.7,-15.9,-0.2,0.0,0.0,0.0
3,A.J. Green,SG,23,MIL,35,1,9.9,1.5,3.6,0.424,...,0.3,0.8,0.111,-0.3,-0.6,-0.9,0.1,0.0,0.0,0.0
4,A.J. Guyton,PG,22,CHI,33,8,19.1,2.4,5.8,0.406,...,-0.1,0.3,0.020,-0.8,-3.2,-4.0,-0.3,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11514,Željko Rebrača,C,29,DET,74,4,15.9,2.6,5.1,0.505,...,1.9,3.3,0.134,-2.9,0.6,-2.3,-0.1,0.0,0.0,0.0
11515,Željko Rebrača,C,30,DET,30,12,16.3,2.7,4.8,0.552,...,0.7,1.3,0.133,-2.8,0.2,-2.6,-0.1,0.0,0.0,0.0
11516,Željko Rebrača,C,31,ATL,24,2,11.4,1.4,3.2,0.442,...,0.4,0.5,0.097,-4.5,0.6,-3.9,-0.1,0.0,0.0,0.0
11517,Željko Rebrača,C,32,LAC,58,2,16.0,2.3,4.0,0.568,...,0.9,2.4,0.122,-1.7,0.4,-1.3,0.2,0.0,0.0,0.0


In [515]:
# Cleaning team data
team_records = team_records.drop(["Unnamed: 0"], axis=1)

In [516]:
team_records["Team"] = team_records["Team"].str.replace("*","",regex=False)
team_records["Team"] = team_records["Team"].str.replace("\((\d+)\)","",regex=True)

In [517]:
team_records["Team"].unique()

array(['Miami Heat', 'New York Knicks', 'Philadelphia 76ers',
       'Orlando Magic', 'Boston Celtics', 'New Jersey Nets',
       'Washington Wizards', 'Indiana Pacers', 'Charlotte Hornets',
       'Toronto Raptors', 'Detroit Pistons', 'Milwaukee Bucks',
       'Cleveland Cavaliers', 'Atlanta Hawks', 'Chicago Bulls',
       'Utah Jazz', 'San Antonio Spurs', 'Minnesota Timberwolves',
       'Dallas Mavericks', 'Denver Nuggets', 'Houston Rockets',
       'Vancouver Grizzlies', 'Los Angeles Lakers',
       'Portland Trail Blazers', 'Phoenix Suns', 'Seattle SuperSonics',
       'Sacramento Kings', 'Golden State Warriors',
       'Los Angeles Clippers', 'Memphis Grizzlies', 'New Orleans Hornets',
       'Charlotte Bobcats', 'New Orleans/Oklahoma City Hornets',
       'Oklahoma City Thunder', 'Brooklyn Nets', 'New Orleans Pelicans',
       'Boston Celtics\xa0', 'Philadelphia 76ers\xa0',
       'New York Knicks\xa0', 'Brooklyn Nets\xa0', 'Toronto Raptors\xa0',
       'Milwaukee Bucks\xa0', 'C

In [518]:
# isolating series and replacing the xa0
xa0_version = team_records["Team"].to_numpy()
new_array = []
for team in xa0_version:
    new_team = team.replace(u'\xa0', "")
    new_array.append(new_team)
# THIS NEW ARRAY IS THE CORRECT TEAM NAMES
#team_records_2023["Team"] = new_array
#team_records_2023
team_records["Team"] = new_array

In [519]:
# Sanity check to make sure the xa0 was removed
team_records["Team"].unique()

array(['Miami Heat', 'New York Knicks', 'Philadelphia 76ers',
       'Orlando Magic', 'Boston Celtics', 'New Jersey Nets',
       'Washington Wizards', 'Indiana Pacers', 'Charlotte Hornets',
       'Toronto Raptors', 'Detroit Pistons', 'Milwaukee Bucks',
       'Cleveland Cavaliers', 'Atlanta Hawks', 'Chicago Bulls',
       'Utah Jazz', 'San Antonio Spurs', 'Minnesota Timberwolves',
       'Dallas Mavericks', 'Denver Nuggets', 'Houston Rockets',
       'Vancouver Grizzlies', 'Los Angeles Lakers',
       'Portland Trail Blazers', 'Phoenix Suns', 'Seattle SuperSonics',
       'Sacramento Kings', 'Golden State Warriors',
       'Los Angeles Clippers', 'Memphis Grizzlies', 'New Orleans Hornets',
       'Charlotte Bobcats', 'New Orleans/Oklahoma City Hornets',
       'Oklahoma City Thunder', 'Brooklyn Nets', 'New Orleans Pelicans'],
      dtype=object)

In [520]:
combined["Tm"].unique()

array(['LAL', 'MIA', 'CLE', 'MIL', 'CHI', 'GSW', 'DAL', 'IND', 'WAS',
       'MIN', 'PHO', 'ATL', 'HOU', 'DEN', 'ORL', 'NOH', 'TOR', 'SAC',
       'CHO', 'PHI', 'BOS', 'OKC', 'NJN', 'NOK', 'LAC', 'UTA', 'CHA',
       'MEM', 'SEA', 'NYK', 'NOP', 'POR', 'BRK', 'DET', 'SAS', 'CHH',
       'VAN'], dtype=object)

In [521]:
nicknames = {}

with open("nicknames.csv") as f:
    lines = f.readlines()
    for line in lines[1:]:
        prefix, name = line.replace("\n", "").split(",")
        nicknames[prefix] = name

In [522]:
combined["Team"] = combined["Tm"].map(nicknames)

/var/folders/64/5vzyghmd6xs0bzvv0yyj64sc0000gr/T/ipykernel_20783/4285917102.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined["Team"] = combined["Tm"].map(nicknames)


In [523]:
stats = combined.merge(team_records, how="outer", on=["Team", "Year"])

In [524]:
stats[stats["Year"] == 2023]

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,Pts Max,Share,Team,W,L,W/L%,GB,PS/G,PA/G,SRS
48,A.J. Green,SG,23,MIL,35,1,9.9,1.5,3.6,0.424,...,0.0,0.000,Milwaukee Bucks,58,24,0.707,—,116.9,113.3,3.61
49,Bobby Portis,PF,27,MIL,70,22,26.0,5.7,11.5,0.496,...,0.0,0.000,Milwaukee Bucks,58,24,0.707,—,116.9,113.3,3.61
50,Brook Lopez,C,34,MIL,78,78,30.4,6.1,11.5,0.531,...,0.0,0.000,Milwaukee Bucks,58,24,0.707,—,116.9,113.3,3.61
51,Giannis Antetokounmpo,PF,28,MIL,63,63,32.1,11.2,20.3,0.553,...,1000.0,0.606,Milwaukee Bucks,58,24,0.707,—,116.9,113.3,3.61
52,Goran Dragić,PG,36,MIL,58,0,15.0,2.4,5.8,0.421,...,0.0,0.000,Milwaukee Bucks,58,24,0.707,—,116.9,113.3,3.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11431,Mitchell Robinson,C,24,NYK,59,58,27.0,3.2,4.7,0.671,...,0.0,0.000,New York Knicks,47,35,0.573,10.0,116.0,113.1,2.99
11432,Obi Toppin,PF,24,NYK,67,5,15.7,2.8,6.3,0.446,...,0.0,0.000,New York Knicks,47,35,0.573,10.0,116.0,113.1,2.99
11433,Quentin Grimes,SG,22,NYK,71,66,29.9,4.0,8.5,0.468,...,0.0,0.000,New York Knicks,47,35,0.573,10.0,116.0,113.1,2.99
11434,RJ Barrett,SG,22,NYK,73,73,33.9,7.0,16.1,0.434,...,0.0,0.000,New York Knicks,47,35,0.573,10.0,116.0,113.1,2.99


In [525]:
stats["GB"] = stats["GB"].str.replace('—', "0.0")
stats["GB"].unique()

array(['0.0', '6.0', '24.0', '37.0', '29.0', '21.0', '28.0', '13.0',
       '25.0', '8.0', '19.0', '3.0', '1.0', '17.0', '23.0', '15.0', '9.0',
       '2.0', '35.0', '20.0', '11.0', '5.0', '26.0', '18.0', '16.0',
       '41.0', '12.0', '14.0', '30.0', '34.0', '4.0', '1.5', '43.0',
       '39.0', '18.5', '31.0', '10.0', '48.0', '40.0', '7.0', '46.0',
       '22.0', '10.5', '32.0', '38.0', '27.0', '33.0', '50.0', '21.5',
       '45.0', '22.5', '25.5', '36.0', '3.5', '20.5', '11.5', '52.0',
       '56.0', '2.5', '12.5', '32.5', '4.5'], dtype=object)

In [489]:
stats["GB"] = pd.to_numeric(stats["GB"])

In [492]:
stats.dtypes

Player     object
Pos        object
Age         int64
Tm         object
G           int64
           ...   
W/L%      float64
GB        float64
PS/G      float64
PA/G      float64
SRS       float64
Length: 61, dtype: object

In [493]:
stats.to_csv("player_mvp_stats.csv")

In [201]:
highest_scoring = stats[stats["G"] > 70].sort_values("PTS", ascending=False)

In [255]:
mvps_df.columns

Index(['Unnamed: 0', 'Rank', 'Player', 'Age', 'Tm', 'First', 'Pts Won',
       'Pts Max', 'Share', 'G', 'MP', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%',
       '3P%', 'FT%', 'WS', 'WS/48', 'Year'],
      dtype='object')